<a href="https://colab.research.google.com/github/josegabrielguerrero26/CSE450-machinelearning/blob/main/project2/starter_housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ** INTRODUCTION**

1. Data Dictionary: https://byui-cse.github.io/cse450-course/module-03/housing-dictionary.txt
2. Problem Intro: https://byui-cse.github.io/cse450-course/module-03/intro.html
3. Dataset https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv

-You’re working as a data analyst for Reddic Housing LLC, a company that specializes in providing house price estimates to development firms.

Recently, a rival company (Grenic Housing Inc) announced that they were launching a set of initiatives to use Artificial Intelligence and Machine Learning to bring their housing estimates into the twenty-first century.

After a heated shareholders meeting, the board of directors has tasked the CEO with making sure Reddic Housing LLC isn’t left behind.

# Discussions Questions


https://byui-cse.github.io/cse450-course/module-03/discussion.html

Questions
You're about to go into a strategy meeting with the CEO, Vice President of Human Resources, and Vice President of Finance. They want to make sure you have the data required to answer the questions they're most interested in.

Be prepared to answer the following questions:

- **PROBLEM TYPE**

DEVON, THE CEO SAYS:
I just sat through four hours of machine learning training with the board of directors this past week, so I'm curious to get your take on this.

Looking at the data and our business model, what kind of machine learning problem do you think we're looking at here?

- **MODEL CONFIDENCE**

CECIL, THE VP OF CUSTOMER RELATIONS ASKS:
My biggest concern right now is making sure that whatever method we come up with to predict housing prices, we can also attach some kind of empirical confidence metric.

Based on your initial analysis of the data, your team feels you can best show confidence in your model by using:

The sum of squares error (SSE).
The mean squared error (MSE).
The root mean squared error (RMSE).
The 2R value.

- **INSURANCE QUESTION**

WILLIAM, THE VP OF FINANCE ASKS:
Our insurance customers are particularly interested in making sure that homes in… unsavory neighborhoods, aren't estimated high.

Is there a way we can easily identify properties in low income areas and have the model lower those estimates to protect our insurance customers' interests?

Based on your initial analysis of the data, your team feels:

We can lower the predicted price for specific neighborhoods before training the model.
We can add average income or other demographic information for the area as features.
For specific zip codes we could add a step to our pipeline that reduces the predicted price by a specific percentage prior to outputting the final price result to ensure the properties aren’t being overvalued.
Taking this kind of action would be a violation of federal laws and/or ethics.

- **DATA ANALYSIS**

JOHNNY, THE DATA SCIENCE INTERN ASKS:
The head of data science says we should use gradient boosted trees for this analysis.

I've noticed that a lot of the features use pretty different ranges.

For example, how should we handle square footage?

# Project Requirements




Overview
After a few more meetings with the executive team, the head of the data science division has assigned your team to address the following issues asked by the stakeholders:


CECIL, THE VP OF CUSTOMER RELATIONS SAYS:
The biggest thing I want to see is quantifiable evidence that the predictions we come up with are reliable.


WILLIAM, THE VP OF FINANCE ASKS:
I'd like to know which property types are weighing most heavily in the house prices predicted by your model. My excel spreadsheets can tell me that information for our current methodology...can your so-called artificial intelligence do the same?


DEVON, THE CEO ADDS:
Yes...thank you William. These are all great questions.

One other question the board was wondering about, is if there are additional factors about these areas that might be affecting prices, which we aren't taking into account.

That may be a little above and beyond what you're team is planning, since it would require finding more data from an external source and correlating it with the data we already have, but if you have the time, I know they'd appreciate it.

If you could send us your team's write up on this by Saturday night, that would be great.


CECIL, THE VP OF CUSTOMER RELATIONS SAYS:
Oh, one more thing. We actually just received a batch of new home data. Could you run your model on it to make some predictions for us? We are anxious to see your model in action.

You can access the data here: https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test.csv

Then, please include with your write up a csv file that has a single column, with the header: "price" and a prediction (one per row) for each of the homes in this file.

TEAM PROJECT EXPECTATIONS

Be sure to read over the Team Project Expectations guide to know what the expectations are for this and future projects.

# Libraries- Dataset

In [1]:
#!pip install ydata_profiling # data Table Analysis
#!pip install typing_extensions # data Table Analysis
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt # for graphs
from sklearn.model_selection import train_test_split #Model Split to train
from sklearn.metrics import accuracy_score           #Accuracy model


dfh = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')

# Table Description

In [3]:
pd.set_option('display.max_columns', None)
dfh.head(5)

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,1565930130,20141104T000000,4,3.25,3760,4675,2.0,0,0,3,8,2740,1020,2007,0,98038,47.3862,-122.048,3280,4033,429900.0
1,3279000420,20150115T000000,3,1.75,1460,7800,1.0,0,0,2,7,1040,420,1979,0,98023,47.3035,-122.382,1310,7865,233000.0
2,194000575,20141014T000000,4,1.00,1340,5800,1.5,0,2,3,7,1340,0,1914,0,98116,47.5658,-122.389,1900,5800,455000.0
3,2115510160,20141208T000000,3,1.75,1440,8050,1.0,0,0,3,8,1440,0,1985,0,98023,47.3187,-122.390,1790,7488,258950.0
4,7522500005,20140815T000000,2,1.50,1780,4750,1.0,0,0,4,7,1080,700,1947,0,98117,47.6859,-122.395,1690,5962,555000.0


In [6]:
dfh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             20000 non-null  int64  
 1   date           20000 non-null  object 
 2   bedrooms       20000 non-null  int64  
 3   bathrooms      20000 non-null  float64
 4   sqft_living    20000 non-null  int64  
 5   sqft_lot       20000 non-null  int64  
 6   floors         20000 non-null  float64
 7   waterfront     20000 non-null  int64  
 8   view           20000 non-null  int64  
 9   condition      20000 non-null  int64  
 10  grade          20000 non-null  int64  
 11  sqft_above     20000 non-null  int64  
 12  sqft_basement  20000 non-null  int64  
 13  yr_built       20000 non-null  int64  
 14  yr_renovated   20000 non-null  int64  
 15  zipcode        20000 non-null  int64  
 16  lat            20000 non-null  float64
 17  long           20000 non-null  float64
 18  sqft_l

# Data Wrangling - Missing Values, transformation

In [7]:
from datetime import date
dfh['date']=  pd.to_datetime(dfh['date'])

In [8]:
dfh.head(5)

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,1565930130,2014-11-04,4,3.25,3760,4675,2.0,0,0,3,8,2740,1020,2007,0,98038,47.3862,-122.048,3280,4033,429900.0
1,3279000420,2015-01-15,3,1.75,1460,7800,1.0,0,0,2,7,1040,420,1979,0,98023,47.3035,-122.382,1310,7865,233000.0
2,194000575,2014-10-14,4,1.00,1340,5800,1.5,0,2,3,7,1340,0,1914,0,98116,47.5658,-122.389,1900,5800,455000.0
3,2115510160,2014-12-08,3,1.75,1440,8050,1.0,0,0,3,8,1440,0,1985,0,98023,47.3187,-122.390,1790,7488,258950.0
4,7522500005,2014-08-15,2,1.50,1780,4750,1.0,0,0,4,7,1080,700,1947,0,98117,47.6859,-122.395,1690,5962,555000.0


# Machine Learning Model

## Split Date

## Imbalance

## Training Model

## Resample

## Prediction

## Evaluation

# Applying Model